Non-bayesian

In [1]:
package_paths = [r"C:\Users\benja\Documents\projects\goalscorers"]
import sys
for path in package_paths:
    sys.path.append(path)
import goalscorer_package.constants as c
import goalscorer_package.data_cleaning as dc
import goalscorer_package.modelling as m
import datetime as dt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import GammaRegressor
# import pymc as pm
# print(f"Running on PyMC v{pm.__version__}")
# import arviz as az
# print(f"Running on Az v{az.__version__}")
# import pytensor.tensor as pt
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import glob
from statistics import mode
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_colwidth", 100)
pd.options.display.float_format = "{: ,.3f}".format
%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 6)
%config InlineBackend.figure_formats = ["retina"]

# Data

In [2]:
seasons = ["2017-2018", "2018-2019"]
comp_ids = [9, 10, 11, 12, 13, 20]

In [3]:
target_variable = "npxg"

In [4]:
def data(seasons: list[str], comp_ids: list[int]) -> pd.DataFrame:
    seasons_leagues = dc.get_seasons_leagues_from_str(seasons, comp_ids)
    df_exp = (
        dc.load_match_expectancies_df(seasons_leagues)
        .assign(date=lambda x: x.datetime.dt.date)
        .drop(columns=["datetime"])
    )
    df = dc.load_seasons_leagues_files("summary", True, seasons_leagues)
    df = dc.split_positions(df)
    df = dc.position_to_generic_position(df)
    df = dc.add_frac_90(df)
    df = dc.add_home(df)
    df = dc.add_opp_team(df)
    df = dc.drop_gk(df)
    df = dc.drop_na_npxg(df)
    df = dc.drop_na_frac_90(df)
    df = dc.add_npg(df)
    df = dc.add_league(df)
    df["season"] = np.vectorize(lambda x: x.season)(df.season_league)
    df = dc.add_datetime(df)
    df = dc.add_expectancies(df, df_exp)
    df = df.dropna(subset=["team_exp"], ignore_index=True)  # drop playoff games
    df = df.assign(
        team_supremacy=lambda x: x.team_exp - x.opp_team_exp,
        npxg=lambda x: np.where(x.npxg == 0.0, 0.0001, x.npxg),
    )
    df = dc.add_match_number(df)
    df = dc.add_t(df, 76)
    return df

In [5]:
def ohe(df_model: pd.DataFrame) -> pd.DataFrame:
    categorical_columns = ["player_id", "opposition_team", "position", "league", "squad"]
    return (
        df_model
        .join(pd.get_dummies(df_model[categorical_columns].astype(str), dtype=int))
        .assign(constant=1)
    )

In [6]:
def create_features(df: pd.DataFrame) -> pd.DataFrame:
    features = [
        "constant",
        "home",
        "ln_frac_90",
        "ln_team_exp",
        "team_supremacy",
        "team_supremacy_2",
    ] + [
        f"player_id_{player_id}" for player_id in df.player_id.unique()
    ] + [
        f"opposition_team_{opposition_team}" for opposition_team in df.opposition_team.unique()
    ] + [
        f"position_{position}" for position in df.position.unique()
    ] + [
        f"league_{league}" for league in df.league.unique()
    ] + [
        f"squad_{squad}" for squad in df.squad.unique()
    ]
    return features

In [7]:
df = data(seasons, comp_ids)

In [8]:
df.head()

,home_team,away_team,datetime,shirtnumber,nationality,position,age,minutes,goals,assists,pens_made,pens_att,shots,shots_on_target,cards_yellow,cards_red,touches,tackles,interceptions,blocks,xg,npxg,xg_assist,sca,gca,passes_completed,passes,passes_pct,progressive_passes,carries,progressive_carries,take_ons,take_ons_won,player_id,player,squad,start,season_league,position_0,position_1,position_2,position_3,position_4,complex_position,frac_90,home,opposition_team,npg,league,season,epoch_datetime,date,team_home,team_away,home_exp,away_exp,success,error,team_exp,opp_team_exp,team_supremacy,match_number,time_interval,t
0,Monaco,Toulouse,2017-08-04 18:45:00,6,fr FRA,CB,24-135,90.000,0,0,0,0,0,0,0,0,32.000,2.000,3,1.000,0.000,0.000,0.000,0.000,0.000,9.000,14.000,64.300,0.000,7.000,0.000,0.000,0.000,f323d995,Christopher Jullien,Toulouse,True,Ligue-1 2017-2018,CB,None,None,None,None,CB,1.000,0,Monaco,0,13,Season 2017-2018 across 2 years,1501872300,2017-08-04,Monaco,Toulouse,2.356,0.709,True,0.000,0.709,2.356,-1.647,1,"(0.999, 56.0]",1
1,Monaco,Toulouse,2017-08-04 18:45:00,20,bf BFA,CB,24-231,90.000,0,0,0,0,0,0,0,0,15.000,0.000,2,2.000,0.000,0.000,0.000,0.000,0.000,7.000,7.000,100.000,0.000,6.000,0.000,1.000,1.000,7f9111ea,Steeve Yago,Toulouse,True,Ligue-1 2017-2018,CB,None,None,None,None,CB,1.000,0,Monaco,0,13,Season 2017-2018 across 2 years,1501872300,2017-08-04,Monaco,Toulouse,2.356,0.709,True,0.000,0.709,2.356,-1.647,1,"(0.999, 56.0]",1
2,Monaco,Toulouse,2017-08-04 18:45:00,27,fr FRA,CM,20-322,90.000,0,0,0,0,0,0,0,0,23.000,3.000,2,2.000,0.000,0.000,0.000,1.000,0.000,11.000,16.000,68.800,1.000,9.000,0.000,0.000,0.000,c69ef855,Alexis Blin,Toulouse,True,Ligue-1 2017-2018,CM,None,None,None,None,CM,1.000,0,Monaco,0,13,Season 2017-2018 across 2 years,1501872300,2017-08-04,Monaco,Toulouse,2.356,0.709,True,0.000,0.709,2.356,-1.647,1,"(0.999, 56.0]",1
3,Monaco,Toulouse,2017-08-04 18:45:00,29,ch SUI,FB,27-044,90.000,0,0,0,0,0,0,1,0,36.000,2.000,0,2.000,0.000,0.000,0.000,0.000,0.000,9.000,24.000,37.500,0.000,12.000,1.000,1.000,1.000,6ca88b2a,François Moubandje,Toulouse,True,Ligue-1 2017-2018,LB,None,None,None,None,LB,1.000,0,Monaco,0,13,Season 2017-2018 across 2 years,1501872300,2017-08-04,Monaco,Toulouse,2.356,0.709,True,0.000,0.709,2.356,-1.647,1,"(0.999, 56.0]",1
4,Monaco,Toulouse,2017-08-04 18:45:00,2,fr FRA,FB,19-177,90.000,0,0,0,0,0,0,0,0,29.000,0.000,1,0.000,0.000,0.000,0.000,0.000,0.000,18.000,21.000,85.700,1.000,7.000,0.000,0.000,0.000,bcc81786,Kelvin Amian,Toulouse,True,Ligue-1 2017-2018,RB,None,None,None,None,RB,1.000,0,Monaco,0,13,Season 2017-2018 across 2 years,1501872300,2017-08-04,Monaco,Toulouse,2.356,0.709,True,0.000,0.709,2.356,-1.647,1,"(0.999, 56.0]",1


In [9]:
df_model = df[["npxg", "npg", "player_id", "home", "opposition_team", "frac_90", "position", "league", "squad", "team_exp", "team_supremacy", "t", 
               "start", "datetime", "season", "home_team", "away_team"]].copy()
df_model = (
    ohe(df_model)
    .assign(
        days_ago=lambda x: (x.datetime.max() - x.datetime).dt.days,
        seasons_ago=lambda x: abs(x.season - x.season.max()).astype(int),
    )
    .assign(
        ln_frac_90=lambda x: np.log(x.frac_90),
        ln_team_exp=lambda x: np.log(x.team_exp),
        team_supremacy_2=lambda x: x.team_supremacy ** 2,
    )
)

df_model.head()

npxg  npg player_id  home opposition_team  frac_90 position  league  \
0  0.000    0  f323d995     0          Monaco    1.000       CB      13   
1  0.000    0  7f9111ea     0          Monaco    1.000       CB      13   
2  0.000    0  c69ef855     0          Monaco    1.000       CM      13   
3  0.000    0  6ca88b2a     0          Monaco    1.000       FB      13   
4  0.000    0  bcc81786     0          Monaco    1.000       FB      13   

      squad  team_exp  team_supremacy  t  start            datetime  \
0  Toulouse     0.709          -1.647  1   True 2017-08-04 18:45:00   
1  Toulouse     0.709          -1.647  1   True 2017-08-04 18:45:00   
2  Toulouse     0.709          -1.647  1   True 2017-08-04 18:45:00   
3  Toulouse     0.709          -1.647  1   True 2017-08-04 18:45:00   
4  Toulouse     0.709          -1.647  1   True 2017-08-04 18:45:00   

                            season home_team away_team  player_id_0000acda  \
0  Season 2017-2018 across 2 years    Monaco  Toulouse                   0   
1  Season 2017-2018 across 2 years    Monaco  Toulouse                   0   
2  Season 2017-2018 across 2 years    Monaco  Toulouse                   0   
3  Season 2017-2018 across 2 years    Monaco  Toulouse                   0   
4  Season 2017-2018 across 2 years    Monaco  Toulouse                   0   

   player_id_000b3da6  player_id_00242715  player_id_00242b75  \
0                   0                   0                   0   
1                   0                   0                   0   
2                   0                   0                   0   
3                   0                   0                   0   
4                   0                   0                   0   

   player_id_0033fb1b  player_id_003a5c72  player_id_00459419  \
0                   0                   0                   0   
1                   0                   0                   0   
2                   0                   0                   0   
3                   0                   0                   0   
4                   0                   0                   0   

   player_id_0049d544  player_id_00528660  player_id_007b39a5  \
0                   0                   0                   0   
1                   0                   0                   0   
2                   0                   0                   0   
3                   0                   0                   0   
4                   0                   0                   0   

   player_id_00823e9a  player_id_00844823  player_id_008a6e6d  \
0                   0                   0                   0   
1                   0                   0                   0   
2                   0                   0                   0   
3                   0                   0                   0   
4                   0                   0                   0   

   player_id_008f854b  player_id_00963611  player_id_00a6485f  \
0                   0                   0                   0   
1                   0                   0                   0   
2                   0                   0                   0   
3                   0                   0                   0   
4                   0                   0                   0   

   player_id_00b0bb87  player_id_00b28772  player_id_00c3d4dc  \
0                   0                   0                   0   
1                   0                   0                   0   
2                   0                   0                   0   
3                   0                   0                   0   
4                   0                   0                   0   

   player_id_00e1cf8f  player_id_0118c54c  player_id_01226327  \
0                   0                   0                   0   
1                   0                   0                   0   
2                   0                   0                   0   
3                   0                   0          

In [10]:
df_model_train, df_model_test = train_test_split(df_model, train_size=0.7, shuffle=False)

# Hyperparameter Tune

In [11]:
def add_weights(df: pd.DataFrame, time_decay_factor: float, season_decay_factor: float) -> pd.DataFrame:
    time_decay = np.exp(-time_decay_factor * df.days_ago)
    season_decay = np.exp(-season_decay_factor * df.seasons_ago)
    game_time_weight = df.frac_90
    # game_time_weight = game_time_factor_1 * df.frac_90 + game_time_factor_2 * (df.frac_90 ** 2)
    return df.assign(weight=time_decay * season_decay * game_time_weight)

In [12]:
def add_ll(df: pd.DataFrame) -> pd.DataFrame:
    return df.assign(
        exponential_ll=lambda x: np.maximum(-10_000.0, np.log(scipy.stats.expon.pdf(x.npxg, scale=x.pred_npxg))),
        poisson_ll=lambda x: np.log(scipy.stats.poisson.pmf(x.npg, mu=x.pred_npxg))
    )

In [13]:
_, df_model_tune_test = train_test_split(df_model_train, train_size=0.7, shuffle=False)
tune_ts = df_model_tune_test.t.unique()

In [15]:
time_decay_factors = [0.01, 0.001, 0.0001]
season_decay_factors = [1.0, 0.3, 0.1, 0.05, 0.01]
regularisation_parameters = [0.001, 0.005, 0.01, 0.05, 0.1]

In [16]:
def predict(ts: list[int], df_model: pd.DataFrame, time_decay_factor: float, season_decay_factor: float, regularisation_parameter: float) -> pd.DataFrame:
    df_test_preds = []

    counter = 1
    # t = ts[0]
    # ts = ts[:2]
    for t in ts:
        # train test split
        df_train, df_test = df_model.query("t < @t"), df_model.query("t == @t")
        
        # add weights
        df_train = add_weights(df_train, time_decay_factor, season_decay_factor)
        
        # standardise train and test
        columns_to_standardise = ["team_exp", "ln_team_exp", "team_supremacy", "team_supremacy_2"]
        scaler = StandardScaler()
        scaler.fit(df_train[columns_to_standardise])
        df_train[columns_to_standardise], df_test[columns_to_standardise] = (
            scaler.transform(df_train[columns_to_standardise]),
            scaler.transform(df_test[columns_to_standardise])
            )
        
        # create X, y
        features = create_features(df_model)
        X_train, X_test = df_train[features], df_test[features]
        y_train, y_test = df_train[target_variable], df_test[target_variable]
        
        # train model
        model = GammaRegressor(max_iter=10_000, fit_intercept=False, alpha=regularisation_parameter)
        model.fit(X_train, y_train)
        
        # test model
        df_test = df_test.assign(pred_npxg=model.predict(X_test))
        df_test_preds.append(df_test[["pred_npxg", "player_id", "home_team", "away_team", "datetime"]])
    
        print("Finished", counter, "/", len(ts))
        counter += 1

    df_test_pred = (
    pd
    .concat(df_test_preds, ignore_index=True)
    .merge(
        df[["player_id", "home_team", "away_team", "datetime", "npxg", "npg"]], 
        how="left", 
        on=["player_id", "home_team", "away_team", "datetime"], 
        validate="1:1"
    )
    )
    df_test_pred = add_ll(df_test_pred)
    return df_test_pred

In [17]:
list_tuning_results = []
for time_decay_factor in time_decay_factors:
    for season_decay_factor in season_decay_factors:
        for regularisation_parameter in regularisation_parameters:
            df_test_pred = predict(tune_ts, df_model_train, time_decay_factor, season_decay_factor, regularisation_parameter)
            list_tuning_results.append({
                "time_decay_factor": time_decay_factor,
                "season_decay_factor": season_decay_factor,
                "regularisation_parameter": regularisation_parameter,
                "exponential_ll": df_test_pred.query("start").exponential_ll.sum(),
                "poisson_ll": df_test_pred.query("start").poisson_ll.sum(),
            })

C:\Users\benja\AppData\Local\Temp\ipykernel_25224\1647004805.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[columns_to_standardise], df_test[columns_to_standardise] = (


Finished 1 / 17


C:\Users\benja\AppData\Local\Temp\ipykernel_25224\1647004805.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[columns_to_standardise], df_test[columns_to_standardise] = (


Finished 2 / 17


C:\Users\benja\AppData\Local\Temp\ipykernel_25224\1647004805.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[columns_to_standardise], df_test[columns_to_standardise] = (


...

Finished 16 / 17


C:\Users\benja\AppData\Local\Temp\ipykernel_25224\1647004805.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[columns_to_standardise], df_test[columns_to_standardise] = (


Finished 17 / 17


C:\Users\benja\AppData\Local\Temp\ipykernel_25224\1276594682.py:3: RuntimeWarning: divide by zero encountered in log
  exponential_ll=lambda x: np.maximum(-10_000.0, np.log(scipy.stats.expon.pdf(x.npxg, scale=x.pred_npxg))),
C:\Users\benja\AppData\Local\Temp\ipykernel_25224\1647004805.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[columns_to_standardise], df_test[columns_to_standardise] = (


KeyboardInterrupt: 

In [ ]:
df_tuning_results = pd.DataFrame(list_tuning_results)

# Fit

In [ ]:
time_decay_factor = 0.01
season_decay_factor = 0.01
regularisation_parameter = 0.01

In [ ]:
# train test split
df_train = df_model.query("t < t.max() + 1")

# add weights
df_train = add_weights(df_train, time_decay_factor, season_decay_factor)

# standardise train and test
columns_to_standardise = ["team_exp", "ln_team_exp", "team_supremacy", "team_supremacy_2"]
scaler = StandardScaler()
scaler.fit(df_train[columns_to_standardise])
df_train[columns_to_standardise], df_test[columns_to_standardise] = (
    scaler.transform(df_train[columns_to_standardise]),
    scaler.transform(df_test[columns_to_standardise])
    )

# create X, y
features = create_features(df_model)
X_train, X_test = df_train[features], df_test[features]
y_train, y_test = df_train[target_variable], df_test[target_variable]

# train model
model = GammaRegressor(max_iter=10_000, fit_intercept=False, alpha=regularisation_parameter)
model.fit(X_train, y_train)